## Homework 2

Instructions
Ваше завдання створити візуалізації, котрі відповідають на наступні питання:

- Як змінювалась структура генерації електроенергії за роками?

- Як залежить споживання електроенергії від дня року та години доби?

- Як змінюється генерація електроенергії з різних джерел впродовж доби?

- Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

- Як змінюється споживання електроенергії впродовж тижня?

Ви самостійно маєте обрати спосіб візуалізації. Ви також маєте написати короткий супровідний текст до кожної візуалізації, котрий пояснює, чому ви обрали саме цей спосіб презентації даних, які ще альтернативи ви розглядали, та чому зупинились саме на цьому варіанті (які його переваги та недоліки).

Складові генерації: АЕС,ТЕЦ,ВДЕ,ТЕС,ГЕС,ГАЕС. Споживання не включає обсяги закачки ГАЕС. Міждержавні перетоки: Україна - Білорусь та РФ, Україна - ЄС, Україна - Молдова. Одиниця виміру - МВт.
Наразі не створено жодного вигляду для цього ресурсу.

In [6]:
import altair as alt
import pandas as pd
alt.renderers.enable('default')

RendererRegistry.enable('default')

## 1. Data investigation

In [8]:
df = pd.read_excel('./2014-2020.xlsx')
df.head()

,Час/Дата,AES,TEC,VDE,TES,GES,GAES_GEN,CONSUMPTION,GAES_PUMP,UK_BLR_RUS,UK_EURO,UK_MLD,Unnamed: 12
0,24-31.12.2020,9235,2039,621,4942,385.0,0,16693,-405.0,-84.0,-11.0,-29.0,NaN
1,23-31.12.2020,9221,2159,707,5549,470.0,0,17805,0.0,-43.0,-212.0,-46.0,NaN
2,22-31.12.2020,9249,2377,709,5906,1000.0,0,18870,0.0,-13.0,-328.0,-30.0,NaN
3,21-31.12.2020,9256,2499,702,6329,909.0,322,19887,0.0,0.0,-64.0,-66.0,NaN
4,20-31.12.2020,9213,2521,702,6640,823.0,602,20387,0.0,-29.0,-48.0,-37.0,NaN


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61368 entries, 0 to 61367
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Час/Дата     61368 non-null  object 
 1   AES          61368 non-null  int64  
 2   TEC          61368 non-null  int64  
 3   VDE          61368 non-null  int64  
 4   TES          61368 non-null  int64  
 5   GES          61367 non-null  float64
 6   GAES_GEN     61368 non-null  int64  
 7   CONSUMPTION  61368 non-null  int64  
 8   GAES_PUMP    61367 non-null  float64
 9   UK_BLR_RUS   61367 non-null  float64
 10  UK_EURO      61367 non-null  float64
 11  UK_MLD       61367 non-null  float64
 12  Unnamed: 12  1 non-null      float64
dtypes: float64(6), int64(6), object(1)
memory usage: 6.1+ MB


## 2. Data preprocessing

## 3. Plotting

### Task 1. Як змінювалась структура генерації електроенергії за роками?

### Task 2. Як залежить споживання електроенергії від дня року та години доби?

### Task 3. Як змінюється генерація електроенергії з різних джерел впродовж доби?

### Task 4. Як змінюється споживання електроенергії впродовж доби у розрізі місяців року та пір року?

### Task 5. Як змінюється споживання електроенергії впродовж тижня?